In [1]:
import prophet as pt
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# Import postgres python adapter
import psycopg2
import logging
import os
import datetime
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
host = os.getenv("HOST")
port = os.getenv("PORT")
database = os.getenv("DATABASE")
user = os.getenv("DB_USER")
password = os.getenv("PASSWORD")

In [3]:
# Read data from postgres
def read_data():
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cur = conn.cursor()
    cur.execute("SELECT * FROM aggregated_demand")
    data = cur.fetchall()
    cur.close()
    conn.close()
    return data

In [4]:
# Convert data to dataframe
def convert_to_dataframe(data):
    df = pd.DataFrame(data, columns=['ds', 'state', 'frequency', 'demand'])
    df['ds'] = pd.to_datetime(df['ds'])
    return df


In [5]:
data = read_data()

In [6]:
data = convert_to_dataframe(data)

In [7]:
# Create a model for each state
states = data['state'].unique()
models = {}

for state in states:
    df = data[data['state'] == state]
    model = pt.Prophet()
    df = df[['ds', 'demand']]
    df.columns = ['ds', 'y']
    model.fit(df)
    models[state] = model


23:40:40 - cmdstanpy - INFO - Chain [1] start processing
23:40:55 - cmdstanpy - INFO - Chain [1] done processing
23:40:56 - cmdstanpy - INFO - Chain [1] start processing
23:41:07 - cmdstanpy - INFO - Chain [1] done processing
23:41:08 - cmdstanpy - INFO - Chain [1] start processing
23:41:22 - cmdstanpy - INFO - Chain [1] done processing
23:41:23 - cmdstanpy - INFO - Chain [1] start processing
23:41:26 - cmdstanpy - INFO - Chain [1] done processing
23:41:27 - cmdstanpy - INFO - Chain [1] start processing
23:41:36 - cmdstanpy - INFO - Chain [1] done processing
23:41:37 - cmdstanpy - INFO - Chain [1] start processing
23:42:00 - cmdstanpy - INFO - Chain [1] done processing
23:42:01 - cmdstanpy - INFO - Chain [1] start processing
23:42:33 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
# create forecast for each state
forecasts = {}
for state in states:
    model = models[state]
    future = model.make_future_dataframe(periods=48*3)
    forecast = model.predict(future)
    forecasts[state] = forecast

In [9]:
# Plot forecast for each state
for state in states:
    forecast = forecasts[state]
    fig = plot_plotly(model, forecast)
    fig.update_layout(
        title=f"Forecast for {state}",
        xaxis_title="Date",
        yaxis_title="Demand (MW)",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
!pip install - -upgrade nbformat



Usage:   
  /usr/local/bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/local/bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/local/bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /usr/local/bin/python3 -m pip install [options] [-e] <local project path> ...
  /usr/local/bin/python3 -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [10]:
import plotly.io as pio

# Plot forecast for each state
for state in states:
    forecast = forecasts[state]
    fig = plot_plotly(model, forecast)
    fig.update_layout(
        title=f"Forecast for {state}",
        xaxis_title="Date",
        yaxis_title="Demand (MW)",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    # Save the plot as an HTML file
    file_name = f"forecast_{state}.html"
    pio.write_html(fig, file_name)
    print(f"Forecast plot for {state} saved as {file_name}")


Forecast plot for Chattishgarh saved as forecast_Chattishgarh.html
Forecast plot for Gujrat saved as forecast_Gujrat.html
Forecast plot for Madhya Pradesh saved as forecast_Madhya Pradesh.html
Forecast plot for Maharashtra saved as forecast_Maharashtra.html
Forecast plot for Goa saved as forecast_Goa.html
Forecast plot for DNH saved as forecast_DNH.html
Forecast plot for DD saved as forecast_DD.html
